In [1]:
import sys
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets
from functools import partial

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils.dataset_tokenization import process_data

In [2]:
model_path = "../../self-corrective-llama_untrained"
# model_name = "MathBite/self_corrective_llama_3.1_8B_untrained"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SelfCorrectiveLlama(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_em

In [3]:
# SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
# INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
# DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
# DELETION_TOKEN_IDS = tokenizer.convert_tokens_to_ids(DELETION_MARKERS)

In [ ]:
# data_path = "../../dataset/train.json"
# dataset = datasets.load_dataset("json", data_files=data_path)

In [5]:
# dataset

In [6]:
# sample = dataset["train"][7295]
# print(sample)
# print(sample["correct_response"])
# print(sample["errors"])
# print(sample["hallucinated_text"])

In [7]:
# for i in range(7295, 7296):
#     sample = dataset["train"][i]
    
#     res = process_data(sample, tokenizer, SPECIAL_INSTRUCTION, INSERTION_MARKER, DELETION_TOKEN_IDS[0], DELETION_TOKEN_IDS[1], DELETION_TOKEN_IDS[2])

#     hall_text_idx = [i for i, label in enumerate(res["hallucination_labels"]) if label == 1]
#     hall_text = [res["input_ids"][i] for i in hall_text_idx]
    
#     print(sample["correct_response"])
#     print("================================================")
#     for error in sample["hallucinated_text"]:
#         print(error)
#     print("================================================")
#     print(tokenizer.decode(hall_text))
#     print("--------------------------------\n\n")

In [8]:
# SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
# INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
# DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
# DELETION_TOKEN_IDS = tokenizer.convert_tokens_to_ids(DELETION_MARKERS)

# mapper = partial(
#     process_data,
#     tokenizer=tokenizer,
#     special_instruction=SPECIAL_INSTRUCTION,
#     insertion_marker=INSERTION_MARKER,
#     del_w_token_id=DELETION_TOKEN_IDS[0],
#     del_s_token_id=DELETION_TOKEN_IDS[1],
#     del_a_token_id=DELETION_TOKEN_IDS[2]
# )

In [9]:
# dataset.cleanup_cache_files()

In [10]:
# tokenized_dataset = dataset.map(mapper, batched=False, load_from_cache_file=False)
# tokenized_dataset = tokenized_dataset["train"]
# columns_to_remove = [
#     "input", "correct_response", "incorrect_response", 
#     "additional_info", "errors", "hallucinated_text"
# ]

# tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)


In [11]:
# tokenized_dataset

In [12]:
# split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
# print(split_dataset)
# train_dataset = split_dataset['train']
# eval_dataset = split_dataset['test']

In [13]:
# output_dir = "../../dataset/training"
# split_dataset.save_to_disk(output_dir)

In [14]:
dataset = datasets.load_from_disk("../../dataset/training")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 36684
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 4077
})


In [28]:
# max_len = 0
# max_len_id = 0
# for i in range(0, len(train_dataset)):
#     sample = train_dataset[i]
#     if len(sample["input_ids"]) > max_len:
#         max_len = len(sample["input_ids"])
#         max_len_id = i

# print(max_len)
# print(max_len_id)


avg_len = 0
for i in range(0, len(train_dataset)):
    sample = train_dataset[i]
    avg_len += len(sample["input_ids"])

print(avg_len / len(train_dataset))

356.67721622505724


In [23]:
# for i in range(4000, 4600):
#     print(len(train_dataset[i]["input_ids"]))

282
257
355
300
535
397
175
322
339
296
326
444
748
791
322
227
273
304
416
283
436
491
455
283
386
239
292
678
328
251
266
368
284
280
431
694
379
289
380
562
274
504
193
534
277
302
627
418
341
614
287
201
229
288
265
240
595
266
279
326
392
660
330
339
419
371
175
397
282
261
341
352
275
543
365
454
327
574
587
430
351
406
295
395
281
383
242
501
321
826
229
378
277
523
347
555
302
487
328
270
235
383
344
283
264
250
346
262
267
385
434
288
467
551
520
339
464
260
167
425
297
569
453
325
473
436
219
657
345
520
229
350
247
363
419
551
316
293
301
509
295
519
826
238
281
296
354
511
532
340
202
283
314
362
311
312
393
309
367
301
279
386
264
371
414
310
481
311
361
275
243
242
412
280
534
297
541
412
398
438
319
323
269
341
402
248
421
585
353
465
276
678
207
187
175
366
175
270
285
398
339
265
307
989
411
320
366
300
369
193
577
498
356
603
453
248
522
281
378
466
310
246
382
278
260
672
406
339
476
423
314
329
381
357
675
532
330
804
241
332
303
322
388
256
457
305
264
334
255
331


In [16]:
1/0

ZeroDivisionError: division by zero

In [ ]:
# import torch

In [ ]:
# sample = train_dataset[5]
# sample["input_ids"] = torch.tensor(sample["input_ids"]).reshape(1, -1)
# sample["hallucination_labels"] = torch.tensor(sample["hallucination_labels"]).reshape(1, -1)
# sample["labels"] = torch.tensor(sample["labels"]).reshape(1, -1)

In [ ]:
# sample

In [ ]:
# print(sample["labels"][sample["labels"] != -100])
# print(sample["hallucination_labels"][sample["hallucination_labels"] != -100])

In [ ]:
# model.forward(
#     input_ids = sample["input_ids"],
#     hallucination_labels = sample["hallucination_labels"],
#     labels = sample["labels"]
# )

In [ ]:
# del_tokens = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]


# for i in range(100, 200):
#     sample = train_dataset[i]
#     print(tokenizer.decode(sample["input_ids"]), "\n")

#     for j in range(1, 4):
#         hall_text_idx = [i for i, label in enumerate(sample["hallucination_labels"]) if label == j]
#         hall_text = [sample["input_ids"][i] for i in hall_text_idx]

#         print("--------------------------------\n")
#         print(f"Deletion token: {del_tokens[j-1]}")
#         print(tokenizer.decode(hall_text))
        
    
#     print("########################################################\n\n")

In [ ]:
# 1/0

In [ ]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     TrainingArguments,
#     BitsAndBytesConfig,
# )
# import torch
# from src.trainer import SelfCorrectionTrainer

In [ ]:
# trainer = SelfCorrectionTrainer(model = "None")

In [ ]:
# sample = train_dataset[0]
# sample["input_ids"] = torch.tensor(sample["input_ids"]).reshape(1, -1)
# sample["hallucination_labels"] = torch.tensor(sample["hallucination_labels"]).reshape(1, -1)
# sample["labels"] = torch.tensor(sample["labels"]).reshape(1, -1)
# print(sample)

In [ ]:
# output = sample.copy()
# # output["logits"] = torch.tensor(output["input_ids"]).reshape(1, -1, 1)
# output["logits"] = torch.zeros(1, output["input_ids"].shape[-1], len(tokenizer.get_vocab())) - 100
# for i in range(output["input_ids"].shape[-1]):
#     output["logits"][0, i, output["input_ids"][0, i]] = 100
# output["hallucination_logits"] = torch.tensor(output["hallucination_labels"]).reshape(1, -1, 1)

In [ ]:
# output["logits"][0, :-1, ...] = output["logits"].clone()[0, 1:, ...]

In [ ]:
# trainer.compute_loss(sample, output, len(tokenizer.get_vocab()))

In [ ]:
# import torch

# logits = torch.zeros(1, 10, 100)
# hallucination_logits = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(1, 10, 1)

# additional_logits = torch.zeros_like(logits)
# additional_logits[:, :, -3:] = hallucination_logits
# logits = logits + additional_logits
# print(logits)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
# base_model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-3.2-1B-Instruct",
# )

In [ ]:
# print(base_model.lm_head.weight.shape)
# print(base_model.lm_head.weight[-5:, :10])

In [ ]:
# base_model.resize_token_embeddings(len(tokenizer.get_vocab())+3)

In [ ]:
# print(base_model.lm_head.weight.shape)
# print(base_model.lm_head.weight[-8:, :10])

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import os
# import sys
# import torch

# # Add the project root directory to the Python path
# project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

In [ ]:
# path = "../../self-corrective-llama_untrained"
# # path = "meta-llama/Llama-3.2-1B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(path)
# tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True)

In [ ]:
model

SelfCorrectiveLlama(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_em

In [ ]:
import torch

def generate_with_hallucination_analysis(model, tokenizer, prompt_text, max_new_tokens=50):
    """
    Generates text and then runs a second pass to get hallucination logits
    for the generated tokens, avoiding interference with the generate loop.
    """
    # Ensure the model is in evaluation mode
    model.eval()
    
    # --- Pass 1: Generate the text ---
    # We use the standard, unmodified generate method.
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # Decode the full generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    
    # --- Pass 2: Run a single forward pass on the full sequence to get all logits ---
    # This is more efficient than collecting them step-by-step.
    with torch.no_grad():
        full_sequence_outputs = model(generated_ids)
    
    # Extract the hallucination logits from the output
    hallucination_logits = full_sequence_outputs.hallucination_logits
    
    # We only care about the logits for the *newly generated* tokens.
    # The logit at position `i` is the prediction for the token at `i+1`.
    prompt_len = inputs.input_ids.shape[1]
    generated_hallucination_logits = hallucination_logits[:, prompt_len-1:-1, :]
    
    return generated_text, generated_hallucination_logits

# --- Example Usage ---
text, hall_logits = generate_with_hallucination_analysis(
    model, 
    tokenizer, 
    "What is the capital of France?",
    max_new_tokens=50
)
print("Generated Text:", text)
print("Hallucination Logits Shape:", hall_logits.shape)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Text: <|begin_of_text|>What is the capital of France? Paris.
The capital of France is Paris. Paris is the most populous city in France and is known for its rich history, art, fashion, and cuisine. It is also home to many famous landmarks such as the Eiffel Tower, Notre Dame
Hallucination Logits Shape: torch.Size([1, 50, 4])


In [ ]:
hall_logits

tensor([[[-0.0483, -0.2782, -0.0229,  0.5821],
         [-0.0457, -0.1654,  0.4881,  0.7241],
         [ 0.1563, -0.2808,  0.1752, -0.0528],
         [ 0.6500, -0.0672,  0.0963,  0.3451],
         [-0.1016,  0.2346,  0.2451,  0.1382],
         [ 0.2472, -0.1517,  0.0213,  0.0206],
         [ 0.3422, -0.2132,  0.6742,  0.3600],
         [ 0.1035,  0.4354, -0.0361,  0.1883],
         [-0.1177, -0.2437,  0.4974,  0.3574],
         [ 0.2063, -0.2982,  0.4049,  0.0723],
         [ 0.2247, -0.1464,  0.2873,  1.2442],
         [-0.0031,  0.1955, -0.2148,  0.1981],
         [ 0.0102, -0.1352,  0.1593, -0.2428],
         [ 0.0284,  0.3141,  0.6227,  0.0378],
         [-0.1079,  0.2176,  0.2375, -0.3359],
         [-0.2790, -0.0070, -0.1074, -0.0306],
         [-0.0129, -0.8086, -0.1504,  0.3837],
         [-0.1234,  0.0691,  0.0285,  0.2415],
         [-0.0553, -0.1397,  0.1646,  0.7801],
         [-0.1584,  0.2697,  0.0933,  0.1162],
         [-0.4042,  0.0693, -0.1235,  0.5240],
         [-0.

In [ ]:
p1 = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a specialized question-answering AI. Your task is to give a concise answer to the question using *only* the provided context. Make sure to always give an answer. Use <DEL_W>, <DEL_S> or <DEL_A> tokens if needed.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext:\n'''\nSichuan has been historically known as the \"Province of Abundance\". It is one of the major agricultural production bases of China. Grain, including rice and wheat, is the major product with output that ranked first in China in 1999. Commercial crops include citrus fruits, sugar cane, sweet potatoes, peaches and grapes. Sichuan also had the largest output of pork among all the provinces and the second largest output of silkworm cocoons in 1999. Sichuan is rich in mineral resources. It has more than 132 kinds of proven underground mineral resources including vanadium, titanium, and lithium being the largest in China. The Panxi region alone possesses 13.3% of the reserves of iron, 93% of titanium, 69% of vanadium, and 83% of the cobalt of the whole country. Sichuan also possesses China's largest proven natural gas reserves, the majority of which is transported to more developed eastern regions.\n'''\n\nQuestion: What are the major agricultural outputs of Sichuan?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
# p1 = "What is the capital of France?"
inputs = tokenizer(p1, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[128000, 128000, 128006,   9125, 128007,    271,   2675,    527,    264,
          28175,   3488,     12,    598,     86,   4776,  15592,     13,   4718,
           3465,    374,    311,   3041,    264,  64694,   4320,    311,    279,
           3488,   1701,    353,   3323,      9,    279,   3984,   2317,     13,
           7557,   2771,    311,   2744,   3041,    459,   4320,     13,   5560,
            220, 128256,     11,    220, 128257,    477,    220, 128258,  11460,
            422,   4460,     13, 128009, 128006,    882, 128007,    271,   2014,
            512,  15029,     50,    718,  10602,    706,   1027,  35901,   3967,
            439,    279,    330,  52174,    315,   3765,  98681,   3343,   1102,
            374,    832,    315,    279,   3682,  29149,   5788,  23963,    315,
           5734,     13,  75374,     11,   2737,  20228,    323,  34153,     11,
            374,    279,   3682,   2027,    449,   2612,    430,  21682,   1176,
            30

In [ ]:
# res = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], output_hallucination_logits=True)
res = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
model.original_vocab_size

128256

In [ ]:
res

tensor([[128000, 128000, 128006,   9125, 128007,    271,   2675,    527,    264,
          28175,   3488,     12,    598,     86,   4776,  15592,     13,   4718,
           3465,    374,    311,   3041,    264,  64694,   4320,    311,    279,
           3488,   1701,    353,   3323,      9,    279,   3984,   2317,     13,
           7557,   2771,    311,   2744,   3041,    459,   4320,     13,   5560,
            220, 128256,     11,    220, 128257,    477,    220, 128258,  11460,
            422,   4460,     13, 128009, 128006,    882, 128007,    271,   2014,
            512,  15029,     50,    718,  10602,    706,   1027,  35901,   3967,
            439,    279,    330,  52174,    315,   3765,  98681,   3343,   1102,
            374,    832,    315,    279,   3682,  29149,   5788,  23963,    315,
           5734,     13,  75374,     11,   2737,  20228,    323,  34153,     11,
            374,    279,   3682,   2027,    449,   2612,    430,  21682,   1176,
            304,   5734,    

In [ ]:
tokens = res

In [ ]:
# tokens, hall = res

In [ ]:
tokenizer.decode(tokens[0])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a specialized question-answering AI. Your task is to give a concise answer to the question using *only* the provided context. Make sure to always give an answer. Use <DEL_W>, <DEL_S> or <DEL_A> tokens if needed.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext:\n\'\'\'\nSichuan has been historically known as the "Province of Abundance". It is one of the major agricultural production bases of China. Grain, including rice and wheat, is the major product with output that ranked first in China in 1999. Commercial crops include citrus fruits, sugar cane, sweet potatoes, peaches and grapes. Sichuan also had the largest output of pork among all the provinces and the second largest output of silkworm cocoons in 1999. Sichuan is rich in mineral resources. It has more than 132 kinds of proven underground mineral resources including vanadium, titanium, and lithium being the largest in China. The P

In [ ]:
1/0

ZeroDivisionError: division by zero

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_name = "meta-llama/Llama-3.2-1B-Instruct"
special_tokens = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
intermediate_save_path = "./temp_resized_model"

# --- Step 1: Load, Resize, and Save the STANDARD Llama model ---
print("--- Step 1: Loading and resizing original LlamaForCausalLM ---")
# Load the original, standard model class
original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
original_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
)

# Add tokens and resize the embeddings of the standard model
original_tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
original_model.resize_token_embeddings(len(original_tokenizer))

# Save this intermediate model. It is still a LlamaForCausalLM, just with a bigger lm_head
print(f"--- Saving resized LlamaForCausalLM to {intermediate_save_path} ---")
original_model.save_pretrained(intermediate_save_path)
original_tokenizer.save_pretrained(intermediate_save_path)

# Clear memory
del original_model
del original_tokenizer
torch.cuda.empty_cache()

--- Step 1: Loading and resizing original LlamaForCausalLM ---


KeyboardInterrupt: 

In [ ]:
# --- Step 2: Load the RESIZED model and run the prompt ---
print("\n--- Step 2: Loading the RESIZED LlamaForCausalLM and testing ---")
# Load the model you just saved. This is still the standard class.
resized_tokenizer = AutoTokenizer.from_pretrained(intermediate_save_path)
resized_model = AutoModelForCausalLM.from_pretrained(
    intermediate_save_path,
)

print(resized_model.lm_head.weight.shape)

prompt = "What is the capital of France?"
inputs = resized_tokenizer(prompt, return_tensors="pt")
outputs = resized_model.generate(**inputs, max_new_tokens=50)
print("--- Output from RESIZED LlamaForCausalLM ---")
print(resized_tokenizer.decode(outputs[0], skip_special_tokens=False))

# Clear memory
del resized_model
del resized_tokenizer
torch.cuda.empty_cache()


--- Step 2: Loading the RESIZED LlamaForCausalLM and testing ---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


torch.Size([128259, 2048])
--- Output from RESIZED LlamaForCausalLM ---
<|begin_of_text|>What is the capital of France? Paris
The capital of France is Paris, which is also the most populous city in the country. Paris is known for its rich history, art, fashion, and cuisine. It is home to many famous landmarks, such as the Eiffel Tower


In [ ]:
# # --- Step 3 (Optional): Confirm your custom model behaves the same ---
# print("\n--- Step 3: Loading the RESIZED model into your custom class ---")
# # This is the final check. We load the same resized model, but now we
# # let it be interpreted as your custom class.
# # We must include the local modeling.py file via `trust_remote_code=True`.
# custom_tokenizer = AutoTokenizer.from_pretrained(intermediate_save_path)
# custom_model = AutoModelForCausalLM.from_pretrained(
#     intermediate_save_path,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True # This will use your local `modeling.py`
# )

# inputs = custom_tokenizer(prompt, return_tensors="pt").to("cuda")
# outputs = custom_model.generate(**inputs, max_new_tokens=50)
# print("--- Output from RESIZED SelfCorrectiveLlama ---")
# print(custom_tokenizer.decode(outputs[0], skip_special_tokens=False))